In [171]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F


cols = ["text", "class"]
df = pd.read_csv('train.csv', encoding='utf-8', names=cols)
df = df.drop(index=0)
df.head()


,text,class
1,thơ lục bát: \n ai ơi xa bến quê hương\nnhớ về...,luc_bat
2,thơ lục bát: \n mùa đông để mộng nằm im\nbao n...,luc_bat
3,thơ lục bát: \n nhớ em sao nhớ thế này\nđi đâu...,luc_bat
4,thơ lục bát: \n một mình qua phố nửa đêm\nđườn...,luc_bat
5,thơ lục bát: \n ở miền biên ải vùng cao\nxa xô...,luc_bat


In [172]:
print(df["class"].unique())

['luc_bat' 'tam_chu' 'bay_chu' 'nam_chu' 'bon_chu']


In [173]:
# Định nghĩa các thẻ bài tương ứng với tiêu đề trong text
TAG_MAPPING = {
    "thơ lục bát:": "<START_LUCBAT>",
    "thơ năm chữ:": "<START_5CHU>",
    "thơ bảy chữ:": "<START_7CHU>",
    "thơ bón chữ:": "<START_4CHU>",
}

def process_text_from_raw(text):
    text = text.strip()
    
    # 1. Tách dòng đầu tiên ra khỏi phần còn lại
    lines = text.split('\n')
    
    if len(lines) < 2:
        return "" # Bỏ qua nếu dữ liệu lỗi (chỉ có tiêu đề hoặc rỗng)
    
    header = lines[0].strip().lower() # Lấy dòng đầu: "thơ lục bát"
    body_lines = lines[1:]            # Lấy các dòng còn lại (nội dung thơ)
    
    # 2. Xác định thẻ bài (Tag)
    # Nếu header có trong từ điển thì lấy, không thì gán nhãn khác
    tag = TAG_MAPPING.get(header, "<START_OTHER>")
    
    # 3. Xử lý phần nội dung thơ
    # Nối các dòng lại, thay dấu xuống dòng bằng <NL>
    # LƯU Ý QUAN TRỌNG: Thêm dấu cách quanh <NL> để tí nữa split() nó tách ra được
    body_text = ' <NL> '.join([line.strip() for line in body_lines])
    
    # 4. Ghép lại thành chuỗi hoàn chỉnh để train
    return f"{tag} {body_text} <END>"
# --- Áp dụng vào DataFrame ---
# Giả sử mày đã load df xong
df['processed_text'] = df['text'].apply(process_text_from_raw)

# Lọc bỏ các dòng bị rỗng do lỗi
df = df[df['processed_text'] != ""]

# Xem thử kết quả 1 dòng
print("--- Dữ liệu gốc ---")
print(df['text'].iloc[1]) 
print("\n--- Dữ liệu sau khi xử lý (Model sẽ học cái này) ---")
print(df['processed_text'].iloc[1])

--- Dữ liệu gốc ---
thơ lục bát: 
 mùa đông để mộng nằm im
bao nhiêu nỗi nhớ biết tìm nơi đâu
trăng bơ vơ lạnh mái đầu
tuyết rơi bông nhớ gió sầu khúc ru

một mình lẻ bóng phiêu du
một mình thổn thức tâm tư quê người
tiếc giọt nắng tuổi đôi mươi
rơi trên thềm mộng cuộc đời trắng trong

nhớ ngọn gió bế tuổi hồng
hương hoa thơm ngát tỏa trong câu thề
để một ngày nhớ dài lê
để nhanh một khắc đêm thề trôi mau

bao nhiêu nỗi nhớ trải dài
đường vu vơ nhớ tay ai dắt mình
bóng cây lặng gió im thinh
tưởng như ai đứng đón mình hôm nao

đông sang để nhạt má đào
tuyết rơi để nhớ đi vào chốn xưa
bây giờ em đã ngủ chưa
có êm chăn ấm gió lùa cô đơn

còn anh đôi mắt chập chờn
cơn mơ chẳng đến dập dồn con tim
vần thơ ao ước mãi tìm
mơ trong nỗi nhớ dáng in hôm nào

--- Dữ liệu sau khi xử lý (Model sẽ học cái này) ---
<START_LUCBAT> mùa đông để mộng nằm im <NL> bao nhiêu nỗi nhớ biết tìm nơi đâu <NL> trăng bơ vơ lạnh mái đầu <NL> tuyết rơi bông nhớ gió sầu khúc ru <NL>  <NL> một mình lẻ bóng phiêu du <N

In [174]:
# Gom toàn bộ text đã xử lý lại
all_text = df['processed_text'].str.cat(sep=' <NL> ')

# Tách từ bằng dấu cách (Word-level Tokenization)
words = all_text.split()

# Tạo bộ từ điển (Vocab)
vocab = sorted(list(set(words)))
vocab_size = len(vocab)
print(f"Vocab size (số lượng từ): {vocab_size}")
# Ví dụ vocab sẽ có: ['<NL>', '<START_LUCBAT>', 'ai', 'bến', 'con', 'hương'...]

# Tạo map
stoi = { w:i for i,w in enumerate(vocab) }
itos = { i:w for i,w in enumerate(vocab) }

# Encoder / Decoder mới
encode = lambda s: [stoi[w] for w in s.split()] 
decode = lambda l: ' '.join([itos[i] for i in l]).replace(' <NL> ', '\n')

# Tạo Tensor data
data = torch.tensor(encode(all_text), dtype=torch.long)

Vocab size (số lượng từ): 16398


In [ ]:

n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [176]:
torch.manual_seed(1337)

batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [177]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [178]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [179]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    

In [180]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [181]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [182]:
import os

def save_checkpoint(model, filename='model_lucbat.pth'):
    """
    Lưu trọng số (weights) của model vào file .pth
    """
    # Lưu state_dict (chỉ lưu tham số, không lưu kiến trúc class)
    torch.save(model.state_dict(), filename)
    print(f"--> Đã lưu model thành công vào file: {filename}")


In [183]:
print(f"Kiểm tra thẻ lục bát: {'<START_LUCBAT>' in stoi}") 
print(f"Kích thước từ điển mới: {vocab_size}")

Kiểm tra thẻ lục bát: True
Kích thước từ điển mới: 16398


In [184]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
best_val_loss = float('inf') 

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # --- THÊM ĐOẠN NÀY ---
        # Nếu loss val thấp hơn kỷ lục cũ thì lưu lại ngay
        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            save_checkpoint(m, 'best_model_lucbat.pth')
            print(f"   (Lưu model mới vì loss giảm xuống {best_val_loss:.4f})")
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
# Muốn nó làm thơ lục bát
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

23.349006 M parameters
step 0: train loss 9.8308, val loss 9.8402
--> Đã lưu model thành công vào file: best_model_lucbat.pth
   (Lưu model mới vì loss giảm xuống 9.8402)
step 500: train loss 4.9289, val loss 5.3266
--> Đã lưu model thành công vào file: best_model_lucbat.pth
   (Lưu model mới vì loss giảm xuống 5.3266)


KeyboardInterrupt: 

In [185]:
context = torch.tensor([encode("<START_LUCBAT>")], dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

def clean_output(text):
    # Thay thế token xuống dòng bằng xuống dòng thật
    text = text.replace(' <NL> ', '\n')
    text = text.replace('<NL>', '\n')

    # Xóa các thẻ bài đi (nhìn cho đỡ rối)
    for tag in ['<START_LUCBAT>', '<START_7CHU>', '<START_5CHU>', '<START_TUDO>', '<START_OTHER>']:
        text = text.replace(tag, '')

    return text.strip()

# Lúc in ra
raw_output = decode(m.generate(context, max_new_tokens=200)[0].tolist())
print(clean_output(raw_output))


<START_LUCBAT> trời mưa mưa lũ mất rồi
so đo ác vọng bao ngày nhọc nhằn
rời đưa mình lại tái tê
tung hcm nước mát vô cùng trời chưa
quảng ninh ngập cả nước mình
ngắm sao nỡ tín còn hơn tháng hai <END>
<START_LUCBAT> chị em xin đừng tạnh hun
luôn là trộn chuyện thình lình em ơi
dù cho chiếu đất em diết
nhớ ngày rau bữa nó nhòa miễn sao
<NL> vua ơi vẫn mãi vẫn còn
con chõng bên ơi thiên đường nhang hao
dậu càng mòn ngẩn lòng mê
nay đào gạo đã bỏ tim biết bao <END>
<START_LUCBAT> tâm hồn bất tử ngẩn ngơ
anh chịu là cái muốn đăng trình thi
thêm thời xưa chung thủy chung
đi qua rồi cũng gian nan đôi tay
chuyến đi tìm kẻ kinh lam
phì nhiêu năm năm qua rồi hôm qua
mong cho em tựa tàu nhau
sớm chiều sưởi ấm người dòng lại già
ngày mai ra đứng lại đây
bao giờ xa cách triền miên để anh <END>
<START_LUCBAT> cái
cô cháu ước đến ngày
hai nhà cứ còn nhớ người em mà thôi
gia đình một chút hoa cau
cùng yêu cha cả lời thầy bình yên <END>
 chiều rơi lá bạn gieo neo
chai thu lá rụng lá rơi ướt mi
theo co